In [2]:
# here we will impor the libraries used for machine learning
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv), data manipulation as in SQL
import matplotlib.pyplot as plt # this is used for the plot the graph 
import seaborn as sns # used for plot interactive graph. I like it most for plot
%matplotlib inline


from sklearn.linear_model import LogisticRegression # to apply the Logistic regression
from sklearn.model_selection import train_test_split, GridSearchCV  # to split the data into two parts
from sklearn.model_selection import KFold, cross_val_score  # use for K- fold cross validation
from sklearn.model_selection import GridSearchCV# for tuning parameter
from sklearn import metrics # for the check the error and accuracy of the model
from sklearn.ensemble import RandomForestClassifier # for random forest classifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm # for Support Vector Machine
from sklearn import metrics # for the check the error and accuracy of the model

from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold

from sklearn.metrics import roc_curve, precision_recall_curve, auc, make_scorer, recall_score, accuracy_score,average_precision_score, precision_score, confusion_matrix, classification_report

In [3]:
from sklearn.model_selection import GridSearchCV# for tuning parameter
import matplotlib.pyplot as plt # this is used for the plot the graph 
import seaborn as sns # used for plot interactive graph. I like it most for plot
%matplotlib inline
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv), data manipulation as in SQL
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score


from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings









warnings.filterwarnings("ignore")
from datetime import datetime

In [4]:
# Importing and Loading the Dataset

dataset = pd.read_csv('NYC Train.csv')
print (dataset.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121995 entries, 0 to 121994
Data columns (total 11 columns):
id                    121995 non-null object
vendor_id             121995 non-null int64
pickup_datetime       121995 non-null object
dropoff_datetime      121995 non-null object
passenger_count       121994 non-null float64
pickup_longitude      121994 non-null float64
pickup_latitude       121994 non-null float64
dropoff_longitude     121994 non-null float64
dropoff_latitude      121994 non-null float64
store_and_fwd_flag    121994 non-null object
trip_duration         121994 non-null float64
dtypes: float64(6), int64(1), object(4)
memory usage: 10.2+ MB
None


In [5]:
print('This dataset contains ',dataset.shape[0],'rows')
print('This dataset contains ',dataset.shape[1],'columns')

('This dataset contains ', 121995, 'rows')
('This dataset contains ', 11, 'columns')


# Handling the Latitude and Longitude features:


## Haversine Distance Between the Two Lat/Lons:
The haversine formula determines the great-circle distance between two points on a sphere given their longitudes and latitudes

In [10]:
def haversine_array(lat1, lng1, lat2, lng2): 
    lat1, lng1, lat2, lng2 = map(np.radians, (lat1, lng1, lat2, lng2)) 
    AVG_EARTH_RADIUS = 6371 # in km 
    lat = lat2 - lat1 
    lng = lng2 - lng1 
    d = np.sin(lat * 0.5) ** 2 + np.cos(lat1) * np.cos(lat2) *      np.sin(lng * 0.5) ** 2 
    h = 2 * AVG_EARTH_RADIUS * np.arcsin(np.sqrt(d)) 
    return h

We could then use the function as:


In [11]:
dataset['haversine_distance'] = dataset.apply(lambda x: haversine_array(x['pickup_latitude'], x['pickup_longitude'], x['dropoff_latitude'], x['dropoff_longitude']),axis=1)


## Manhattan Distance Between the two Lat/Lons:
The distance between two points measured along axes at right angles

In [12]:
def dummy_manhattan_distance(lat1, lng1, lat2, lng2): 
    a = haversine_array(lat1, lng1, lat1, lng2) 
    b = haversine_array(lat1, lng1, lat2, lng1) 
    return a + b

We could then use the function as:


In [13]:
dataset['manhattan_distance'] = dataset.apply(lambda x: dummy_manhattan_distance(x['pickup_latitude'], x['pickup_longitude'], x['dropoff_latitude'], x['dropoff_longitude']),axis=1)


## Finding the Bearing Between the two Lat/Lons:
A bearing is used to represent the direction of one point relative to another point

In [14]:
def bearing_array(lat1, lng1, lat2, lng2): 
    AVG_EARTH_RADIUS = 6371 # in km 
    lng_delta_rad = np.radians(lng2 - lng1) 
    lat1, lng1, lat2, lng2 = map(np.radians, (lat1, lng1, lat2, lng2)) 
    y = np.sin(lng_delta_rad) * np.cos(lat2) 
    x = np.cos(lat1) * np.sin(lat2) - np.sin(lat1) * np.cos(lat2) * np.cos(lng_delta_rad) 
    return np.degrees(np.arctan2(y, x))


We could then use the function as:


In [15]:
dataset['bearing'] = dataset.apply(lambda x: bearing_array(x['pickup_latitude'], x['pickup_longitude'], x['dropoff_latitude'], x['dropoff_longitude']),axis=1)


## Finding the Center Latitude and Longitude between Pickup and Dropoff:


In [16]:
dataset.loc[:, 'center_latitude'] = (dataset['pickup_latitude'].values + dataset['dropoff_latitude'].values) / 2 
dataset.loc[:, 'center_longitude'] = (dataset['pickup_longitude'].values + dataset['dropoff_longitude'].values) / 2


### These are the new columns that we create:

In [17]:
dataset.head(3)

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,log_trip_duration,haversine_distance,manhattan_distance,bearing,center_latitude,center_longitude
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1.0,-73.982155,40.767937,-73.964630,40.765602,N,455.0,6.122493,1.498521,1.735433,99.970196,40.766769,-73.973392
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1.0,-73.980415,40.738564,-73.999481,40.731152,N,663.0,6.498282,1.805507,2.430506,-117.153768,40.734858,-73.989948
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1.0,-73.979027,40.763939,-74.005333,40.710087,N,2124.0,7.661527,6.385098,8.203575,-159.680165,40.737013,-73.992180


### log transformation:


In [5]:
dataset['log_trip_duration'] = dataset['trip_duration'].apply(lambda x: np.log(1+x))

In [19]:
dataset.head(3)

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,log_trip_duration,haversine_distance,manhattan_distance,bearing,center_latitude,center_longitude
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1.0,-73.982155,40.767937,-73.964630,40.765602,N,455.0,6.122493,1.498521,1.735433,99.970196,40.766769,-73.973392
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1.0,-73.980415,40.738564,-73.999481,40.731152,N,663.0,6.498282,1.805507,2.430506,-117.153768,40.734858,-73.989948
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1.0,-73.979027,40.763939,-74.005333,40.710087,N,2124.0,7.661527,6.385098,8.203575,-159.680165,40.737013,-73.992180
